## Ingesting PDF

In [17]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
# local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

# Local PDF file uploads

loader = UnstructuredPDFLoader(file_path=r'F:\0 - Bibliotecas Windows\Área de trabalho\biddings-genai\database\CHECK-LIST - LICITAÇÕES.pdf')
data = loader.load()


# loader = DirectoryLoader(r'F:\0 - Bibliotecas Windows\Área de trabalho\biddings-genai\database', glob="./*.docx", loader_cls=UnstructuredWordDocumentLoader)
# documents = loader.load()

In [19]:
# # Preview first page
# data[0].page_content

data

[Document(metadata={'source': 'F:\\0 - Bibliotecas Windows\\Área de trabalho\\biddings-genai\\database\\CHECK-LIST - LICITAÇÕES.pdf'}, page_content='Belém, 16 de fevereiro de 2024.\n\nI.\n\nÀ DOCUMENTOS CHECK-LIST PARTICIPAÇÃO LICITATÓRIOS DE PROCESSOS CONTRATAÇÃO ADMINISTRATIVA (COM FULCRO NAQUILO QUE DETERMINA A LEI Nº 14.133/21, ART. 62 e ss.:\n\nDOS\n\nNECESSÁRIOS\n\nEM\n\nHabilitação Jurídica (art. 66)\n\nAto Constitutivo (contrato social, estatuto social ou requerimento de empresário) - em vigor devidamente inscrito na Junta Comercial e todas as suas alterações. Em caso de Sociedades Comerciais por ações, deverá ser apresentado acompanhado de ata de eleição de seus administradores e, para Sociedades Civis, deve ser acompanhado de prova de diretoria em exercício. Para Empresa Individual, é o Registro Comercial;\n\nTodas as alterações ou consolidação do Ato Constitutivo; • Procuração dos respectivos representantes nas licitações; • Documentos dos Sócios; • Documentos do Represent

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=200)  # fine-tuning

texts = text_splitter.split_documents(data)
texts


[Document(metadata={'source': 'F:\\0 - Bibliotecas Windows\\Área de trabalho\\biddings-genai\\database\\CHECK-LIST - LICITAÇÕES.pdf'}, page_content='Belém, 16 de fevereiro de 2024.\n\nI.\n\nÀ DOCUMENTOS CHECK-LIST PARTICIPAÇÃO LICITATÓRIOS DE PROCESSOS CONTRATAÇÃO ADMINISTRATIVA (COM FULCRO NAQUILO QUE DETERMINA A LEI Nº 14.133/21, ART. 62 e ss.:\n\nDOS\n\nNECESSÁRIOS\n\nEM\n\nHabilitação Jurídica (art. 66)\n\nAto Constitutivo (contrato social, estatuto social ou requerimento de empresário) - em vigor devidamente inscrito na Junta Comercial e todas as suas alterações. Em caso de Sociedades Comerciais por ações, deverá ser apresentado acompanhado de ata de eleição de seus administradores e, para Sociedades Civis, deve ser acompanhado de prova de diretoria em exercício. Para Empresa Individual, é o Registro Comercial;\n\nTodas as alterações ou consolidação do Ato Constitutivo; • Procuração dos respectivos representantes nas licitações; • Documentos dos Sócios; • Documentos do Represent

## Vector Embeddings

In [21]:
!ollama pull nomic-embed-text 

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [22]:
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pull

In [23]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
mistral:latest         	2ae6f6dd7a3d	4.1 GB	Less than a second ago	
nomic-embed-text:latest	0a109f422b47	274 MB	8 minutes ago         	


In [24]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [25]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=texts, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 4/4 [01:22<00:00, 20.71s/it]


In [26]:
vector_db.embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='nomic-embed-text', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=True, headers=None, model_kwargs=None)

## Retrieval

In [27]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [28]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [29]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [30]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [31]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [32]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.62s/it]


' Hello! In this context, it appears to be a checklist for participating in bidding processes for administrative contracts. The list includes various documents and declarations required such as Habilitação Jurídica (Article 66), Habilitação Social, Fiscal and Trabalhista (Article 68), Qualificação Econômico-Financeira (Article 69), Qualificação Técnica (Article 67) among others. The individual listed, Igor Lamartine Nogueira Auad, is an OAB/PA lawyer with number 14.921. The location and date are Belém, 16 de fevereiro de 2024.'

In [33]:
chain.invoke("What are the 5 pillars of global cooperation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.63s/it]


' The context provided does not contain information about the 5 pillars of global cooperation.'

In [34]:
# Delete all collections in the db
vector_db.delete_collection()